In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import sys
import tempfile
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from torch_points3d.metrics.colored_tqdm import Coloredtqdm as Ctq
from tqdm.std import tqdm
import torch_points3d.core.data_transform as T3D
import torch_geometric.transforms as T
import torch.utils.data as data
from torch_geometric.data import Batch
from torch_points3d.datasets.batch import SimpleBatch
import torch
import torch.utils.data as data
import numpy as np
import random
from torch_geometric.data import Batch, Data

In [ ]:
yaml_config = """
conv_type: "SPARSE"
define_constants:
    in_feat: 128
    block: ResBlock # Can be any of the blocks in modules/SparseConv3d/modules.py
down_conv:
    module_name: ResNetDown
    block: block
    N: [0, 1, 2, 2, 3]
    down_conv_nn:
        [
            [FEAT, in_feat],
            [in_feat, in_feat],
            [in_feat, 2*in_feat],
            [2*in_feat, 4*in_feat],
            [4*in_feat, 8*in_feat],
        ]
    kernel_size: [3, 3, 3, 3, 3]
    stride: [1, 2, 2, 2, 2]
innermost:
    module_name: GlobalBaseModule
    activation:
        name: LeakyReLU
        negative_slope: 0.2
    aggr: "max"
    nn: [8*in_feat, 8*in_feat]
""" 

from omegaconf import OmegaConf
params = OmegaConf.create(yaml_config)

In [ ]:
from torch_points3d.applications.sparseconv3d import SparseConv3d

class spConvregress(torch.nn.Module):
    def __init__(self):
        super().__init__() 
        self.encoder = SparseConv3d("encoder", input_nc=1, num_layers=2, output_nc=230, config=params) # minkowski by default
        self.log_softmax = torch.nn.LogSoftmax(dim=-1)
    def forward(self, data):
        # Set labels for the tracker
        
        # Forward through the network
        data_out = self.encoder(data)
        self.output = self.log_softmax(data_out.x.squeeze())
        return self.output

In [ ]:
model = spConvregress().cuda()

In [ ]:
model

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir=f'/home/ssd1/shkim/logs35/icsd_230class_sp3_rand_128feat_4')

In [ ]:
import pickle
with open("ICSD_labels.pickle" , "rb") as f:
    dic = pickle.load(f)

In [ ]:
import glob
data_list = []
pt_list = glob.glob("/home/ssd1/shkim/icsd_cart_rand_pt/*.pt")
for i in tqdm(pt_list):
    data = torch.load(i)
    data.tar = dic[i.split('/')[-1].split('.')[0]] - 1 # if in order to use 7class and 101 class  comment on  -1
    data_list.append(data)

In [ ]:
# if data list make to training 7 class
# run previous cell and run this cell
for i in dic:
    i.tar= dic[i]
    if i.tar== 1 or i.tar== 2:
        dic[i] = torch.tensor(0, dtype=torch.int32)
    elif i.tar>= 3 and i.tar<= 15:
        dic[i] = torch.tensor(1, dtype=torch.int32)
    elif val_c>= 16 and i.tar<= 74:
        dic[i] = torch.tensor(2, dtype=torch.int32)
    elif i.tar>= 75 and i.tar<= 142:
        dic[i] = torch.tensor(3, dtype=torch.int32)
    elif val_c>= 143 and i.tar<= 167:
        dic[i] = torch.tensor(4, dtype=torch.int32)
    elif i.tar>= 168 and i.tar<= 194:
        dic[i] = torch.tensor(5, dtype=torch.int32)
    elif i.tar>= 195 and i.tar<= 230:
        dic[i] = torch.tensor(6, dtype=torch.int32)       

In [ ]:
# if data list make to training 101 class
# run previous cell of load pt file and run this cell
for i in data_list:

    if i.tar + 1 in (1,2):
        i.tar = 1            
    elif i.tar + 1 in (3,6,10):
        i.tar = 2            
    elif i.tar + 1 in (4,11):
        i.tar = 3  
    elif i.tar + 1 in (5,8,12):
        i.tar = 4  
    elif i.tar + 1 in (7,13):
        i.tar = 5          
    elif i.tar + 1 in (9,15):
        i.tar = 6            
    elif i.tar + 1 == 14:
        i.tar = 7  
    elif i.tar + 1 in (16,25,47):
        i.tar = 8  
    elif i.tar + 1 == 17:
        i.tar = 9            
    elif i.tar + 1 == 18:
        i.tar = 10            
    elif i.tar + 1 == 19:
        i.tar = 11  
    elif i.tar + 1 == 20:
        i.tar = 12  
    elif i.tar + 1 in (21,35,38,65):
        i.tar = 13            
    elif i.tar + 1 in (22,42,69):
        i.tar = 14            
    elif i.tar + 1 in (23,24,44,71):
        i.tar = 15  
    elif i.tar + 1 in (26,28,51):
        i.tar = 16  
    elif i.tar + 1 in (27,49):
        i.tar = 17            
    elif i.tar + 1 in (29,57):
        i.tar = 18            
    elif i.tar + 1 in (30,53):
        i.tar = 19            
    elif i.tar + 1 in (31,59):
        i.tar = 20  
    elif i.tar + 1 in (32,55):
        i.tar = 21  
    elif i.tar + 1 in (33,62):
        i.tar = 22  
    elif i.tar + 1 in (34,58):
        i.tar = 23            
    elif i.tar + 1 in (36,40,63):
        i.tar = 24            
    elif i.tar + 1 in (37,66):
        i.tar = 25  
    elif i.tar + 1 in (39,67):
        i.tar = 26  
    elif i.tar + 1 in (41,64):
        i.tar = 27          
    elif i.tar + 1 == 43:
        i.tar = 28            
    elif i.tar + 1 in (45,72):
        i.tar = 29            
    elif i.tar + 1 in (46,74):
        i.tar = 30  
    elif i.tar + 1 == 48:
        i.tar = 31  
    elif i.tar + 1 == 50:
        i.tar = 32  
    elif i.tar + 1 == 52:
        i.tar = 33            
    elif i.tar + 1 == 54:
        i.tar = 34            
    elif i.tar + 1 == 56:
        i.tar = 35  
    elif i.tar + 1 == 60:
        i.tar = 36  
    elif i.tar + 1 == 61:
        i.tar = 37
    elif i.tar + 1 == 68:
        i.tar = 38            
    elif i.tar + 1 == 70:
        i.tar = 39            
    elif i.tar + 1 == 73:
        i.tar = 40  
    elif i.tar + 1 in (75,81,83,89,99,111,115,123):
        i.tar = 41        
    elif i.tar + 1 in (76,78,91,95):
        i.tar = 42  
    elif i.tar + 1 in (77,84,93):
        i.tar = 43            
    elif i.tar + 1 in (79,82,87,97,107,119,121,139):
        i.tar = 44            
    elif i.tar + 1 in (80,98):
        i.tar = 45  
    elif i.tar + 1 in (85,129):
        i.tar = 46  
    elif i.tar + 1 == 86:
        i.tar = 47
    elif i.tar + 1 == 88:
        i.tar = 48            
    elif i.tar + 1 in (90,113):
        i.tar = 49            
    elif i.tar + 1 in (92,96):
        i.tar = 50  
    elif i.tar + 1 == 94:
        i.tar = 51        
    elif i.tar + 1 in (100,117,127):
        i.tar = 52  
    elif i.tar + 1 in (101,116,132):
        i.tar = 53            
    elif i.tar + 1 in (102,118,136):
        i.tar = 54            
    elif i.tar + 1 in (103,124):
        i.tar = 55  
    elif i.tar + 1 in (104,128):
        i.tar = 56  
    elif i.tar + 1 in (105,112,131):
        i.tar = 57
    elif i.tar + 1 in (106,135):
        i.tar = 58            
    elif i.tar + 1 in (108,120,140):
        i.tar = 59            
    elif i.tar + 1 in (109,122):
        i.tar = 60  
    elif i.tar + 1 == 110:
        i.tar = 61  
    elif i.tar + 1 == 114:
        i.tar = 62  
    elif i.tar + 1 == 125:
        i.tar = 63            
    elif i.tar + 1 == 126:
        i.tar = 64            
    elif i.tar + 1 == 130:
        i.tar = 65  
    elif i.tar + 1 == 133:
        i.tar = 66  
    elif i.tar + 1 == 134:
        i.tar = 67
    elif i.tar + 1 == 137:
        i.tar = 68            
    elif i.tar + 1 == 138:
        i.tar = 69            
    elif i.tar + 1 == 141:
        i.tar = 70          
    elif i.tar + 1 == 142:
        i.tar = 71          
    elif i.tar + 1 in (143,147,149,150,156,157,162,164):
        i.tar = 72  
    elif i.tar + 1 in (144,145,151,152,153,154):
        i.tar = 73            
    elif i.tar + 1 in (146,148,155,160,166):
        i.tar = 74            
    elif i.tar + 1 in (158,165):
        i.tar = 75  
    elif i.tar + 1 in (159,163):
        i.tar = 76  
    elif i.tar + 1 in (161,167):
        i.tar = 77
    elif i.tar + 1 in (168,174,175,177,183,187,189,191):
        i.tar = 78            
    elif i.tar + 1 in (169,170,178,179):
        i.tar = 79            
    elif i.tar + 1 in (171,172,180,181):
        i.tar = 80    
    elif i.tar + 1 in (173,176,182):
        i.tar = 81          
    elif i.tar + 1 in (184,192):
        i.tar = 82  
    elif i.tar + 1 in (185,188,193):
        i.tar = 83            
    elif i.tar + 1 in (186,190,194):
        i.tar = 84            
    elif i.tar + 1 in (195,200,207,215,221):
        i.tar = 85  
    elif i.tar + 1 in (196,202,209,216,225):
        i.tar = 86  
    elif i.tar + 1 in (197,199,204,211,217,229):
        i.tar = 87
    elif i.tar + 1 in (198,208):
        i.tar = 88            
    elif i.tar + 1 in (201,224):
        i.tar = 89            
    elif i.tar + 1 in (203,227):
        i.tar = 90          
    elif i.tar + 1 == 205:
        i.tar = 91          
    elif i.tar + 1 == 206:
        i.tar = 92  
    elif i.tar + 1 == 210:
        i.tar = 93            
    elif i.tar + 1 in (212,213):
        i.tar = 94            
    elif i.tar + 1 == 214:
        i.tar = 95  
    elif i.tar + 1 in (218,223):
        i.tar = 96  
    elif i.tar + 1 in (219,226):
        i.tar = 97
    elif i.tar + 1 == 220:
        i.tar = 98            
    elif i.tar + 1 == 222:
        i.tar = 99            
    elif i.tar + 1 == 228:
        i.tar = 100  
    elif i.tar + 1 == 230:
        i.tar = 101 
        
for i in data_list:
    i.tar = i.tar - 1

In [ ]:
datall = []
for i in tqdm(data_list):
    if torch.isinf(i.x.max()) == False:
        datall.append(i)

In [ ]:
data_train, test_files = train_test_split(datall, test_size=0.1, random_state=35)
train_files, val_files = train_test_split(data_train, test_size=0.13333, random_state=35)

In [ ]:
class SaveBestModel:
    """
    Class to save the best model while training. If the current epoch's 
    validation loss is less than the previous least less, then save the
    model state.
    """
    def __init__(
        self, best_valid_loss=0
    ):
        self.best_valid_loss = best_valid_loss
        
    def __call__(
        self, current_valid_loss, 
        epoch, model, optimizer
    ):
        if current_valid_loss > self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, f'/home/ssd1/shkim/logs35/icsd_230class_sp3_rand_128feat_4/best_model.pth')

save_best_model = SaveBestModel()


In [ ]:
num_epochs = 3000

collate_function = lambda datalist: Batch.from_data_list(datalist)
train_dataloader = torch.utils.data.DataLoader(
train_files, 
batch_size=64, 
shuffle=True, 
num_workers=0,
drop_last=True,
collate_fn=collate_function, pin_memory = True , 
)

collate_function = lambda datalist: Batch.from_data_list(datalist)
val_dataloader = torch.utils.data.DataLoader(
val_files, 
batch_size=64, 
shuffle=False, 
num_workers=0,  
collate_fn=collate_function, pin_memory = True, #sampler=val_sampler
)
import torch.nn as nn



collate_function = lambda datalist: Batch.from_data_list(datalist)
test_dataloader = torch.utils.data.DataLoader(
test_files, 
batch_size=64, 
shuffle=False, 
num_workers=0,  
collate_fn=collate_function, pin_memory = True, #sampler=val_sampler
)
import torch.nn as nn



In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm
optimizer = optim.Adam(model.parameters(), lr=7.8125e-06) #0.0005)
#optimizer = optim.SGD(ddp_model.parameters(), lr=0.03, momentum=0.9, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)
#loss_function =torch.nn.functional.nll_loss().cuda()

# Training loop
for epoch in range(3000):
    # Train model
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    train_acc_t = 0
    model.train()
    n = 1
  

    with tqdm(train_dataloader, unit="batch", total=len(train_dataloader)) as tepoch:
        for batch_data in tepoch:

            batch_data = batch_data.cuda().to(torch.float32)
            optimizer.zero_grad()
            logits = model(batch_data)
            loss = torch.nn.functional.nll_loss(logits.squeeze(), batch_data.tar.to(torch.long))

            loss.backward()
            # 가상 배치 크기에 도달하면, 그래디언트 업데이트

            optimizer.step()
            
            train_loss += loss.item()
            labels_again = torch.argmax(logits, dim=1)
            correct = torch.eq(batch_data.tar, labels_again).sum().item() # torch.eq() calculates where two tensors are equal
            train_acc_t += (correct / len(logits)) * 100 
            tepoch.set_postfix(total_loss=train_loss/n, batch_loss=loss.item(), train_acc = train_acc_t/n)
            n = n + 1
            
        train_loss /= len(train_dataloader)
        train_acc_t /= len(train_dataloader)

        print(f" Train Loss: loss {train_loss:.4f}")
        print(f" Train acc: acc {train_acc_t:.4f}")
        writer.add_scalar(f"Loss/train", train_loss, epoch)
        writer.add_scalar(f"acc/train", train_acc_t, epoch)
        

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    val_acc_t = 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(tqdm(val_dataloader)):
            batch_data = batch_data.cuda().to(torch.float32)
            logits = model(batch_data)


            loss = torch.nn.functional.nll_loss(logits.squeeze(), batch_data.tar.to(torch.long))

            val_loss += loss.item()
            labels_again = torch.argmax(logits, dim=1)
            correct = torch.eq(batch_data.tar, labels_again).sum().item() # torch.eq() calculates where two tensors are equal
            val_acc_t += (correct / len(logits)) * 100 

        val_loss /= len(val_dataloader)
        val_acc_t /= len(val_dataloader)
        scheduler.step(val_loss)

        save_best_model(val_acc_t, epoch, model, optimizer)

        writer.add_scalars("every_1_epoch", {
                                "Loss/train_2":train_loss,
                                "acc/train_2":train_acc_t,
                                "Loss/val_2":val_loss,
                                "acc/val_2":val_acc_t,
                                "learning_Rate":scheduler.optimizer.param_groups[0]['lr']},epoch)

        print(f"Validation Loss:  loss {val_loss:.4f}")
        print(f"Validation acc:  acc {val_acc_t:.4f}")
        model_test(model, test_dataloader)

In [ ]:
model.load_state_dict(torch.load("/home/ssd1/shkim/logs22/mp_7class_sp3/best_model.pth")['model_state_dict'])

In [ ]:
from tqdm.auto import tqdm
model.eval()
val_loss = 0.0
val_correct = 0
val_total = 0
val_acc_t = 0
real = []
pred = []
with torch.no_grad():
    for batch_idx, batch_data in enumerate(tqdm(test_dataloader)):

        batch_data = batch_data.cuda().to(torch.float32)
        logits = model(batch_data)


        loss = torch.nn.functional.nll_loss(logits.squeeze(), batch_data.tar.to(torch.long))

        val_loss += loss.item()
        labels_again = torch.argmax(logits.squeeze(), dim=1)
        pred.append(labels_again)
        real.append(batch_data.tar)
        correct = torch.eq(batch_data.tar, labels_again).sum().item() # torch.eq() calculates where two tensors are equal
        val_acc_t += (correct / len(logits)) * 100 

    val_loss /= len(test_dataloader)
    val_acc_t /= len(test_dataloader)
    

    print(f"Validation Loss:  loss {val_loss:.4f}")
    print(f"Validation acc:  acc {val_acc_t:.4f}")

In [ ]:
real_list = []
for i in real:
    for j in range(i.size()[0]):
        real_list.append(i[j].item())

In [ ]:
pred_list = []
for i in pred:
    for j in range(i.size()[0]):
        pred_list.append(i[j].item())

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(real_list, pred_list))
print(f1_score(real_list, pred_list, average='macro'))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(real_list, pred_list, normalize='true')